In [70]:
import sklearn
import pandas as pd
import boto3
import os
from sagemaker import get_execution_role
import sagemaker
import json
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.parameters import (
    ParameterInteger, 
    ParameterFloat, 
    ParameterString, 
    ParameterBoolean
)

from sagemaker.processing import (
    ProcessingInput, 
    ProcessingOutput, 
    ScriptProcessor
)

from sagemaker.workflow.steps import (
    ProcessingStep, 
    TrainingStep, 
    CreateModelStep
)
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.parameters import (
    ParameterInteger, 
    ParameterFloat, 
    ParameterString, 
    ParameterBoolean
)
from sagemaker.workflow.clarify_check_step import (
    ModelBiasCheckConfig, 
    ClarifyCheckStep, 
    ModelExplainabilityCheckConfig
)
from sagemaker import Model
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.conditions import (
    ConditionGreaterThan,
    ConditionGreaterThanOrEqualTo
)

from sagemaker.inputs import TrainingInput
from sagemaker.estimator import Estimator
from sagemaker.workflow.steps import TrainingStep

from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

In [71]:
import pandas as pd
import json
import boto3
import pathlib
import io
import sagemaker
from time import gmtime, strftime, sleep
from sagemaker.deserializers import CSVDeserializer
from sagemaker.serializers import CSVSerializer

from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import (
    ProcessingInput, 
    ProcessingOutput, 
    ScriptProcessor
)
from sagemaker.inputs import TrainingInput

from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import (
    ProcessingStep, 
    TrainingStep, 
    CreateModelStep
)
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.parameters import (
    ParameterInteger, 
    ParameterFloat, 
    ParameterString, 
    ParameterBoolean
)
from sagemaker.workflow.clarify_check_step import (
    ModelBiasCheckConfig, 
    ClarifyCheckStep, 
    ModelExplainabilityCheckConfig
)
from sagemaker import Model
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.conditions import (
    ConditionGreaterThan,
    ConditionGreaterThanOrEqualTo
)
from sagemaker.workflow.pipeline_experiment_config import PipelineExperimentConfig
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import (
    Join,
    JsonGet
)
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)
from sagemaker.lambda_helper import Lambda

from sagemaker.model_metrics import (
    MetricsSource, 
    ModelMetrics, 
    FileSource
)
from sagemaker.drift_check_baselines import DriftCheckBaselines

from sagemaker.image_uris import retrieve


sagemaker.__version__

'2.168.0'

## Set constants

In [72]:
# Get some variables you need to interact with SageMaker service
boto_session = boto3.Session()
region = boto_session.region_name
bucket_name = sagemaker.Session().default_bucket()
bucket_prefix = "Solar-Power-Forcasting/linear"  
sm_session = sagemaker.Session()
sm_client = boto_session.client("sagemaker")
sm_role = sagemaker.get_execution_role()

initialized = True

print(sm_role)

arn:aws:iam::531485126105:role/service-role/AmazonSageMaker-ExecutionRole-20230614T171444


In [73]:
# # Store some variables to keep the value between the notebooks
%store bucket_name
%store bucket_prefix
%store sm_role
%store region
%store initialized

Stored 'bucket_name' (str)
Stored 'bucket_prefix' (str)
Stored 'sm_role' (str)
Stored 'region' (str)
Stored 'initialized' (bool)


In [74]:
import json

NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"

try:
    with open(NOTEBOOK_METADATA_FILE, "r") as f:
        data = json.load(f)
        print(data)
except FileNotFoundError:
    print("File not found.")


{'AppType': 'KernelGateway', 'DomainId': 'd-1qvmpqvqiuve', 'UserProfileName': 'default-1686740036091', 'ResourceArn': 'arn:aws:sagemaker:us-east-1:531485126105:app/d-1qvmpqvqiuve/default-1686740036091/KernelGateway/datascience-1-0-ml-t3-medium-1abf3407f667f989be9d86559395', 'ResourceName': 'datascience-1-0-ml-t3-medium-1abf3407f667f989be9d86559395', 'AppImageVersion': ''}


In [75]:
%store -r domain_id
print(domain_id)

d-1qvmpqvqiuve


In [76]:
# Set names of pipeline objects
project = "solar1"

pipeline_name = f"{project}-pipeline"
pipeline_model_name = f"{project}-model-reg"
model_package_group_name = f"{project}-model-group"
endpoint_config_name = f"{project}-endpoint-config"
endpoint_name = f"{project}-endpoint"

# Set instance types and counts
process_instance_type = "ml.c5.xlarge"
train_instance_count = 1
train_instance_type = "ml.m5.xlarge"

# Set S3 urls for processed data
train_s3_url = f"s3://{bucket_name}/{bucket_prefix}/train"
validation_s3_url = f"s3://{bucket_name}/{bucket_prefix}/validation"
test_s3_url = f"s3://{bucket_name}/{bucket_prefix}/test"
baseline_s3_url = f"s3://{bucket_name}/{bucket_prefix}/baseline"

evaluation_s3_url = f"s3://{bucket_name}/{bucket_prefix}/evaluation"
prediction_baseline_s3_url = f"s3://{bucket_name}/{bucket_prefix}/prediction_baseline"

output_s3_url = f"s3://{bucket_name}/{bucket_prefix}/output"

In [77]:
output_s3_url

's3://sagemaker-us-east-1-531485126105/Solar-Power-Forcasting/linear/output'

In [78]:
# #store the variable
%store train_s3_url
%store validation_s3_url
%store test_s3_url
%store baseline_s3_url
%store model_package_group_name
%store evaluation_s3_url
%store prediction_baseline_s3_url
%store output_s3_url

Stored 'train_s3_url' (str)
Stored 'validation_s3_url' (str)
Stored 'test_s3_url' (str)
Stored 'baseline_s3_url' (str)
Stored 'model_package_group_name' (str)
Stored 'evaluation_s3_url' (str)
Stored 'prediction_baseline_s3_url' (str)
Stored 'output_s3_url' (str)


In [79]:
# Set instance types and counts
process_instance_type = "ml.c5.xlarge"
train_instance_count = 1
train_instance_type = "ml.m5.xlarge"

In [94]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

Stored variables and their in-db values:
athena_table_name                      -> 'sagemaker_workshop_e2e_churn_1686747619'
baseline_s3_url                        -> 's3://sagemaker-us-east-1-531485126105/Solar-Power
bucket                                 -> 'sagemaker-studio-us-east-1-531485126105'
bucket_name                            -> 'sagemaker-us-east-1-531485126105'
bucket_prefix                          -> 'Solar-Power-Forcasting/linear'
churn_feature_group_name               -> 'sagemaker-workshop-e2e-churn'
docker_image_name                      -> '683313688378.dkr.ecr.us-east-1.amazonaws.com/sage
domain_id                              -> 'd-1qvmpqvqiuve'
evaluation_s3_url                      -> 's3://sagemaker-us-east-1-531485126105/Solar-Power
experiment_name                        -> 'Solar-Power-experiment-25-10-00-09'
framework_version                      -> '1.3-1'
initialized                            -> True
input_s3_url                           -> 's3://sagem

## Read data and upload it 

In [81]:
#read data and save it in pandas dataframe
df_gen1 = pd.read_csv("data/Plant_1_Generation_Data.csv")
df_gen2 = pd.read_csv("data/Plant_2_Generation_Data.csv")

df_weather1 = pd.read_csv("data/Plant_1_Weather_Sensor_Data.csv")
df_weather2 = pd.read_csv("data//Plant_2_Weather_Sensor_Data.csv")

In [82]:
df_gen1.head()

          DATE_TIME  PLANT_ID       SOURCE_KEY  DC_POWER  AC_POWER  \
0  15-05-2020 00:00   4135001  1BY6WEcLGh8j5v7       0.0       0.0   
1  15-05-2020 00:00   4135001  1IF53ai7Xc0U56Y       0.0       0.0   
2  15-05-2020 00:00   4135001  3PZuoBAID5Wc2HD       0.0       0.0   
3  15-05-2020 00:00   4135001  7JYdWkrLSPkdwr4       0.0       0.0   
4  15-05-2020 00:00   4135001  McdE0feGgRqW7Ca       0.0       0.0   

   DAILY_YIELD  TOTAL_YIELD  
0          0.0    6259559.0  
1          0.0    6183645.0  
2          0.0    6987759.0  
3          0.0    7602960.0  
4          0.0    7158964.0  

In [83]:
# Run this cell to import or install the Data Wrangler widget to show automatic visualization and generate code to fix data quality issues
try:
    import sagemaker_datawrangler
except ImportError:
    !pip install --upgrade sagemaker-datawrangler
    import sagemaker_datawrangler

# Display Pandas DataFrame to view the widget: df, display(df), df.sample()... 

In [96]:
#create S3 bucket 
# !aws s3 mb s3://sagemaker-eu-central-1-d48
# bucket_name='sagemaker-eu-central-1-d48'
bucket_prefix='solar1/linerreg'


In [85]:
#Upload data to S3 bucket
s3 = boto3.client('s3')

# # Upload Plant_1_Generation_Data.csv
# s3.upload_file("data/Plant_1_Generation_Data.csv", bucket_name, "input/Plant_1_Generation_Data.csv")

# # Upload Plant_2_Generation_Data.csv
# s3.upload_file("data/Plant_2_Generation_Data.csv", bucket_name, "input/Plant_2_Generation_Data.csv")

# # Upload Plant_1_Weather_Sensor_Data.csv
# s3.upload_file("data/Plant_1_Weather_Sensor_Data.csv", bucket_name, "input/Plant_1_Weather_Sensor_Data.csv")

# # Upload Plant_2_Weather_Sensor_Data.csv
# s3.upload_file("data/Plant_2_Weather_Sensor_Data.csv", bucket_name, "input/Plant_2_Weather_Sensor_Data.csv")

# print("Upload complete.")

In [86]:
try:
    input_s3_url
except NameError:      
    # If input_s3_url is not defined, upload the datasets to S3 and store the paths
    input_s3_url_gen1 = sagemaker.Session().upload_data(
        path="data/Plant_1_Generation_Data.csv",
        bucket=bucket_name,
        key_prefix=f"{bucket_prefix}/input"
    )
    input_s3_url_gen2 = sagemaker.Session().upload_data(
        path="data/Plant_2_Generation_Data.csv",
        bucket=bucket_name,
        key_prefix=f"{bucket_prefix}/input"
    )
    input_s3_url_weather1 = sagemaker.Session().upload_data(
        path="data/Plant_1_Weather_Sensor_Data.csv",
        bucket=bucket_name,
        key_prefix=f"{bucket_prefix}/input"
    )
    input_s3_url_weather2 = sagemaker.Session().upload_data(
        path="data/Plant_2_Weather_Sensor_Data.csv",
        bucket=bucket_name,
        key_prefix=f"{bucket_prefix}/input"
    )
    print("Upload complete.")

    # %store input_s3_url_gen1
    # %store input_s3_url_gen2
    # %store input_s3_url_weather1
    # %store input_s3_url_weather2

In [87]:
# check how many file in s3

response = s3.list_objects_v2(Bucket=bucket_name)

if 'Contents' in response:
    file_count = len(response['Contents'])
    print(f"Number of files in {bucket_name}: {file_count}")
else:
    print(f"No files found in {bucket_name}")

Number of files in sagemaker-us-east-1-531485126105: 282


In [88]:
#check all file name in s3

response = s3.list_objects_v2(Bucket=bucket_name)

if 'Contents' in response:
    print(f"Files in {bucket_name}:")
    for obj in response['Contents']:
        file_name = obj['Key']
        print(file_name)
else:
    print(f"No files found in {bucket_name}")

Files in sagemaker-us-east-1-531485126105:
assignment_day_4/bank-additional-full.csv
athena/athena/780924dbaf2149e5990163cb452fcae6/data/20230614_130819_00020_cddss_11abd477-1de8-45b8-951c-c3780c2716c9
athena/athena/780924dbaf2149e5990163cb452fcae6/data/20230614_130819_00020_cddss_1351986c-9010-4ba2-80b3-bd6502143f3d
athena/athena/780924dbaf2149e5990163cb452fcae6/data/20230614_130819_00020_cddss_16305422-5d01-4837-9379-0f0c1797aecd
athena/athena/780924dbaf2149e5990163cb452fcae6/data/20230614_130819_00020_cddss_2017370c-f8ca-4e38-8e41-198235d716c6
athena/athena/780924dbaf2149e5990163cb452fcae6/data/20230614_130819_00020_cddss_3799497a-7792-4a37-a304-925b56ac7d61
athena/athena/780924dbaf2149e5990163cb452fcae6/data/20230614_130819_00020_cddss_4be6b5af-aecd-46fe-b0bc-2c2fd0d57f8d
athena/athena/780924dbaf2149e5990163cb452fcae6/data/20230614_130819_00020_cddss_52a50cb2-9b5e-47cb-94b7-82815a7c85e7
athena/athena/780924dbaf2149e5990163cb452fcae6/data/20230614_130819_00020_cddss_5c2280dd-8daa-48

## Create pipeline
### Setup pipeline parameters

In [89]:
# Set processing instance type
process_instance_type_param = ParameterString(
    name="ProcessingInstanceType",
    default_value=process_instance_type,
)

# Set training instance type
train_instance_type_param = ParameterString(
    name="TrainingInstanceType",
    default_value=train_instance_type,
)

# Set training instance count
train_instance_count_param = ParameterInteger(
    name="TrainingInstanceCount",
    default_value=train_instance_count
)

# Set model approval param
model_approval_status_param = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval"
)

# Minimal threshold for model performance on the test dataset
test_score_threshold_param = ParameterFloat(
    name="TestScoreThreshold", 
    default_value=0.75
)

# Set S3 url for input dataset
input_s3_url_param_g1 = ParameterString(
    name="InputDataUrgen1",
    default_value=input_s3_url_gen1,
)

input_s3_url_param_g2 = ParameterString(
    name="InputDataUrgen2",
    default_value=input_s3_url_gen2,
)

input_s3_url_param_w1= ParameterString(
    name="InputDataUrweather1",
    default_value=input_s3_url_weather1,
)

input_s3_url_param_w2 = ParameterString(
    name="InputDataUrweather2",
    default_value=input_s3_url_weather2,
)

NameError: name 'input_s3_url_gen1' is not defined

### Build the pipeline steps


In [ ]:
session = PipelineSession()

#### Processing step

In [ ]:
df_weather1.columns

In [ ]:
df_gen1.columns

In [ ]:
session = PipelineSession()

In [ ]:
%%writefile preprocessing.py

import pandas as pd
import numpy as np
import argparse
import os

def _parse_args():
    
    parser = argparse.ArgumentParser()
    # Data, model, and output directories
    # model_dir is always passed in from SageMaker. By default, this is an S3 path under the default bucket.
    parser.add_argument('--filepath', type=str, default='/opt/ml/processing/input/')
    parser.add_argument('--filename', type=str, default='input')
    parser.add_argument('--outputpath', type=str, default='/opt/ml/processing/output/')
    
    return parser.parse_known_args()


if __name__=="__main__":
    # Process arguments
    args, _ = _parse_args()
    
    target_col = "DC_POWER"
    
    # adding separate time and date columns
    combined_plant["DATE"] = pd.to_datetime(combined_plant["DATE_TIME"]).dt.date # add new column with date
    combined_plant["TIME"] = pd.to_datetime(combined_plant["DATE_TIME"]).dt.time # add new column with time
    combined_plant["MONTH"] = pd.to_datetime(combined_plant["DATE_TIME"]).dt.month # add new column with month

    # add hours and minutes for ml models
    combined_plant['HOURS'] = pd.to_datetime(combined_plant['TIME'],format='%H:%M:%S').dt.hour
    combined_plant['MINUTES'] = pd.to_datetime(combined_plant['TIME'],format='%H:%M:%S').dt.minute
    combined_plant['MINUTES_PASS'] = combined_plant['MINUTES'] + combined_plant['HOURS']*60

    
    # Drop unnecessary columns and merge dataframes
    df_plant1 = pd.merge(
        df_gen1.drop(columns=['PLANT_ID','AC_POWER','DAILY_YIELD']),
        df_weather1.drop(columns=['PLANT_ID', 'SOURCE_KEY']),
        on='DATE_TIME'
    )
    
    df_plant2 = pd.merge(
        df_gen2.drop(columns=['PLANT_ID','AC_POWER','DAILY_YIELD']),
        df_weather2.drop(columns=['PLANT_ID', 'SOURCE_KEY']),
        on='DATE_TIME'
    )
    
    combined_plant = pd.concat([df_plant1, df_plant2])
    
    # adding separate time and date columns
    combined_plant["DATE"] = pd.to_datetime(combined_plant["DATE_TIME"]).dt.date # add new column with date
    combined_plant["TIME"] = pd.to_datetime(combined_plant["DATE_TIME"]).dt.time # add new column with time

    # add hours and minutes for ml models
    combined_plant['HOURS'] = pd.to_datetime(combined_plant['TIME'],format='%H:%M:%S').dt.hour
    combined_plant['MINUTES'] = pd.to_datetime(combined_plant['TIME'],format='%H:%M:%S').dt.minute
    combined_plant['MINUTES_PASS'] = df_plant1['MINUTES'] + combined_plant['HOURS']*60

    data_folder = "/data"  # Specify the data folder path

    # Check if the data folder exists; if not, create it
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)

    combined_plant_file = os.path.join(data_folder, "combined_plant.csv")

    combined_plant.to_csv(combined_plant_file, index=False)
    print(f"combined_plant.csv saved to {combined_plant_file}")
    
    # Shuffle and split the dataset
    train_data, validation_data, test_data = np.split(
        combined_plant.sample(frac=1, random_state=1729),
        [int(0.7 * len(df_plant)), int(0.9 * len(df_plant))],
    )

    print(f"Data split > train:{train_data.shape} | validation:{validation_data.shape} | test:{test_data.shape}")
    
    # Save datasets locally
    train_data.to_csv(os.path.join(args.outputpath, 'train/train.csv'), index=False, header=False)
    validation_data.to_csv(os.path.join(args.outputpath, 'validation/validation.csv'), index=False, header=False)
    test_data[target_col].to_csv(os.path.join(args.outputpath, 'test/test_y.csv'), index=False, header=False)
    test_data.drop([target_col], axis=1).to_csv(os.path.join(args.outputpath, 'test/test_x.csv'), index=False, header=False)


    # Save the baseline dataset for model monitoring
    combined_plant.drop([target_col], axis=1).to_csv(os.path.join(args.outputpath, 'baseline/baseline.csv'), index=False, header=False)
    
    print("## Processing complete. Exiting.")


In [90]:
df_gen1 = pd.read_csv(os.path.join("data", 'Plant_1_Generation_Data.csv'))
df_gen2 = pd.read_csv(os.path.join("data", 'Plant_2_Generation_Data.csv'))
df_weather1 = pd.read_csv(os.path.join("data", 'Plant_1_Weather_Sensor_Data.csv'))
df_weather2 = pd.read_csv(os.path.join("data", 'Plant_2_Weather_Sensor_Data.csv'))

# Adjust datetime format
df_gen1['DATE_TIME'] = pd.to_datetime(df_gen1['DATE_TIME'])
df_weather1['DATE_TIME'] = pd.to_datetime(df_weather1['DATE_TIME'])
df_gen2['DATE_TIME'] = pd.to_datetime(df_gen2['DATE_TIME'])
df_weather2['DATE_TIME'] = pd.to_datetime(df_weather2['DATE_TIME'])

# Drop unnecessary columns and merge dataframes
df_plant1 = pd.merge(
    df_gen1.drop(columns=['PLANT_ID','AC_POWER','DAILY_YIELD']),
    df_weather1.drop(columns=['PLANT_ID', 'SOURCE_KEY']),
    on='DATE_TIME'
)

df_plant2 = pd.merge(
    df_gen2.drop(columns=['PLANT_ID','AC_POWER','DAILY_YIELD']),
    df_weather2.drop(columns=['PLANT_ID', 'SOURCE_KEY']),
    on='DATE_TIME'
)

combined_plant = pd.concat([df_plant1, df_plant2])

# adding separate time and date columns
combined_plant["DATE"] = pd.to_datetime(combined_plant["DATE_TIME"]).dt.date # add new column with date
combined_plant["TIME"] = pd.to_datetime(combined_plant["DATE_TIME"]).dt.time # add new column with time
combined_plant["MONTH"] = pd.to_datetime(combined_plant["DATE_TIME"]).dt.month # add new column with month

# add hours and minutes for ml models
combined_plant['HOURS'] = pd.to_datetime(combined_plant['TIME'],format='%H:%M:%S').dt.hour
combined_plant['MINUTES'] = pd.to_datetime(combined_plant['TIME'],format='%H:%M:%S').dt.minute
combined_plant['MINUTES_PASS'] = combined_plant['MINUTES'] + combined_plant['HOURS']*60


data_folder = "data"  # Specify the data folder path

# Define the file path
file_path = os.path.join(data_folder, "combined_plant.csv")

combined_plant.to_csv(file_path, index=False)
print("File saved successfully.")

File saved successfully.


In [91]:
combined_plant.head()\

   DATE_TIME       SOURCE_KEY  DC_POWER  TOTAL_YIELD  AMBIENT_TEMPERATURE  \
0 2020-05-15  1BY6WEcLGh8j5v7       0.0    6259559.0            25.184316   
1 2020-05-15  1IF53ai7Xc0U56Y       0.0    6183645.0            25.184316   
2 2020-05-15  3PZuoBAID5Wc2HD       0.0    6987759.0            25.184316   
3 2020-05-15  7JYdWkrLSPkdwr4       0.0    7602960.0            25.184316   
4 2020-05-15  McdE0feGgRqW7Ca       0.0    7158964.0            25.184316   

   MODULE_TEMPERATURE  IRRADIATION        DATE      TIME  MONTH  HOURS  \
0           22.857507          0.0  2020-05-15  00:00:00      5      0   
1           22.857507          0.0  2020-05-15  00:00:00      5      0   
2           22.857507          0.0  2020-05-15  00:00:00      5      0   
3           22.857507          0.0  2020-05-15  00:00:00      5      0   
4           22.857507          0.0  2020-05-15  00:00:00      5      0   

   MINUTES  MINUTES_PASS  
0        0             0  
1        0             0  
2        0 

In [ ]:
s3 = boto3.client('s3')

# Check if the file exists
if os.path.isfile(file_path):
    # Upload the file to S3
    s3.upload_file(file_path, bucket_name, "input/combined_plant.csv")
    print('File uploaded successfully.')
else:
    print('File does not exist.')

# Upload Plant_1_Generation_Data.csv
s3.upload_file(file_path, bucket_name, "input/combined_plant.csv")
print('File uploaded successfully.')

# !python preprocessing.py

In [ ]:
# This cell is tagged with `parameters` tag and will be overwritten if the notebook executed headlessly
file_source = "EFS"
file_name = "bank-additional-full.csv"
input_path = "./data/bank-additional" 
output_path = "./data"

In [ ]:
# filename = "baseline.csv"

# # If running interactively, upload data to S3 to have it here for a headless run
# if file_source == 'EFS':
#     input_s3_url = session.upload_data(
#         path=os.path.join(input_path, file_name),
#         bucket=bucket_name,
#         key_prefix=f"{bucket_prefix}/input"
#     )
    
#     print(input_s3_url)

In [92]:
experiment_name = f"Solar-Power-experiment-{strftime('%d-%H-%M-%S', gmtime())}"
%store experiment_name

Stored 'experiment_name' (str)


In [93]:
experiment_name

'Solar-Power-experiment-25-10-00-09'

In [ ]:
# Create SKLearnProcessor
sklearn_processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=sm_role,
    instance_type=process_instance_type_param.default_value,
    instance_count=1,
    base_job_name=f"{pipeline_name}/preprocess",
    sagemaker_session=session,
)


# Define processing inputs
processing_inputs = [
    ProcessingInput(source=input_s3_url_param_g1, destination="/opt/ml/processing/input"),
    ProcessingInput(source=input_s3_url_param_g2, destination="/opt/ml/processing/input"),
    ProcessingInput(source=input_s3_url_param_w1, destination="/opt/ml/processing/input"),
    ProcessingInput(source=input_s3_url_param_w2, destination="/opt/ml/processing/input"),
]

# Define processing outputs
processing_outputs = [
    ProcessingOutput(output_name="train_data", source="/opt/ml/processing/output/train", destination=train_s3_url),
    ProcessingOutput(output_name="validation_data", source="/opt/ml/processing/output/validation", destination=validation_s3_url),
    ProcessingOutput(output_name="test_data", source="/opt/ml/processing/output/test", destination=test_s3_url),
    ProcessingOutput(output_name="baseline_data", source="/opt/ml/processing/output/baseline", destination=baseline_s3_url),
]

# Run the SKLearnProcessor
processor_args = sklearn_processor.run(
    inputs=processing_inputs,
    outputs=processing_outputs,
    code='preprocessing.py',
    # arguments = ['arg1', 'arg2'],
)

# Define processing step
step_process = ProcessingStep(
    name=f"{pipeline_name}-preprocess-data",
    step_args=processor_args,
)


# If running interactively, upload data to S3 to have it here for a headless run
if file_source == 'EFS':
    input_s3_url = session.upload_data(
        path=os.path.join(input_path, file_name),
        bucket=bucket_name,
        key_prefix=f"{bucket_prefix}/input"
    )
    
    print(input_s3_url)#### Training step

In [ ]:
LinearRegression_image_uri = sagemaker.image_uris.retrieve(framework='linear-learner',region=region)

In [ ]:
# Instantiate a Linear Learner estimator object
estimator = Estimator(
    image_uri=LinearRegression_image_uri,
    role=get_execution_role(),
    instance_type=train_instance_type_param,
    instance_count=train_instance_count_param,
    output_path=output_s3_url,
    sagemaker_session=session,
    base_job_name=f"{pipeline_name}/train",
)

# Define algorithm hyperparameters
estimator.set_hyperparameters(
    predictor_type='regressor',  # Set the predictor type to 'regressor' for linear regression
    feature_dim=5,  # Specify the number of input features
    epochs=100,  # Number of training epochs
    mini_batch_size=32,  # Mini-batch size for training
    learning_rate=0.1,  # Learning rate for the optimizer
    normalize_data='true',  # Normalize the input features
    loss='squared_loss',  # Loss function for linear regression
    early_stopping_patience=10,  # Patience for early stopping
    early_stopping_tolerance=0.001,  # Tolerance for early stopping
    early_stopping_enabled=True,  # Enable early stopping
)

# Define training inputs
training_inputs = {
    "train": TrainingInput(
        s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train_data"].S3Output.S3Uri,
        content_type="text/csv",
    ),
    "validation": TrainingInput(
        s3_data=step_process.properties.ProcessingOutputConfig.Outputs["validation_data"].S3Output.S3Uri,
        content_type="text/csv",
    ),
}

# Fit the estimator to the training data
training_args = estimator.fit(training_inputs)

# Define the training step
step_train = TrainingStep(
    name=f"{pipeline_name}-train",
    step_args=training_args,
)


#### Evaluation step
Create a model evaluation script to check if the model performance meets the specified threshold. 

In [ ]:
# Set processing instance type
process_instance_type_param = ParameterString(
    name="ProcessingInstanceType",
    default_value=process_instance_type,
)

# Set training instance type
train_instance_type_param = ParameterString(
    name="TrainingInstanceType",
    default_value=train_instance_type,
)

# Set training instance count
train_instance_count_param = ParameterInteger(
    name="TrainingInstanceCount",
    default_value=train_instance_count
)

# Set model approval param
model_approval_status_param = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval"
)

# Minimal threshold for model performance on the test dataset
test_score_threshold_param = ParameterFloat(
    name="TestScoreThreshold", 
    default_value=0.75
)

# Set S3 url for input dataset
input_s3_url_param = ParameterString(
    name="InputDataUrl",
    default_value=input_s3_url,
)

In [ ]:
%%writefile evaluation.py

import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

if __name__ == "__main__":   
    
    # All paths are local for the processing container
    model_path = "/opt/ml/processing/model/model.tar.gz"
    test_x_path = "/opt/ml/processing/test/test_x.csv"
    test_y_path = "/opt/ml/processing/test/test_y.csv"
    output_dir = "/opt/ml/processing/evaluation"
    output_prediction_path = "/opt/ml/processing/output/"
        
    # Read model tar file
    with tarfile.open(model_path, "r:gz") as t:
        t.extractall(path=".")
    
    # Load model
    model = linear.Booster()
    model.load_model("linear-model")
    
    # Read test data
    X_test = pd.read_csv(test_x_path, header=None).values
    y_test = pd.read_csv(test_y_path, header=None).to_numpy()

    # Run predictions
    predictions = model.predict(X_test)

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    
    # Calculate R2 score
    r2 = r2_score(y_test, predictions)
    
    report_dict = {
        "regression_metrics": {
            "rmse": {
                "value": rmse,
            },
            "r2_score": {
                "value": r2,
            },
        },
    }

    # Save evaluation report
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)
    with open(f"{output_dir}/evaluation.json", "w") as f:
        f.write(json.dumps(report_dict))
    
    # Save prediction baseline file - we need it later for the model quality monitoring
    pd.DataFrame({"prediction": predictions,
                  "label": y_test.squeeze()}
                ).to_csv(os.path.join(output_prediction_path, 'prediction_baseline/prediction_baseline.csv'), index=False, header=True)


Create a processor to run the evaluation script and construct the evaluation step:

In [ ]:
# script_processor = ScriptProcessor(
#     image_uri=LinearRegression_image_uri,
#     role=sm_role,
#     command=["python3"],
#     instance_type=process_instance_type_param,
#     instance_count=1,
#     base_job_name=f"{pipeline_name}/evaluate",
#     sagemaker_session=session,
# )

# .......................

In [ ]:
script_processor = ScriptProcessor(
    image_uri=LinearRegression_image_uri,
    role=sm_role,
    command=["python3"],
    instance_type=process_instance_type_param,
    instance_count=1,
    base_job_name=f"{pipeline_name}/evaluate",
    sagemaker_session=session,
)

eval_inputs=[
    ProcessingInput(source=step_train.properties.ModelArtifacts.S3ModelArtifacts, 
                    destination="/opt/ml/processing/model"),
    ProcessingInput(source=step_process.properties.ProcessingOutputConfig.Outputs["test_data"].S3Output.S3Uri, 
                    destination="/opt/ml/processing/test"),
]

eval_outputs=[
    ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation", 
                     destination=evaluation_s3_url),
    ProcessingOutput(output_name="prediction_baseline_data", source="/opt/ml/processing/output/prediction_baseline", 
                     destination=prediction_baseline_s3_url),
]

eval_args = script_processor.run(
    inputs=eval_inputs,
    outputs=eval_outputs,
    code="evaluation.py",
)
    
evaluation_report = PropertyFile(
    name="ModelEvaluationReport", output_name="evaluation", path="evaluation.json"
)

step_eval = ProcessingStep(
    name=f"{pipeline_name}-evaluate-model",
    step_args=eval_args,
    property_files=[evaluation_report]
)

#### Register step
The register step creates a SageMaker model and registers a new version of a model in the SageMaker Model Registry within a [model package group](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-model-group.html).

In [ ]:
# model = Model(
#     image_uri=LinearRegression_image_uri,        
#     model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
#     name=f"Solar-energy-regression-model",
#     sagemaker_session=session,
#     role=sm_role,
# )

# ....................................

In [ ]:
model = Model(
    image_uri=LinearRegression_image_uri,        
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    name=f"from-idea-to-prod-linear-model",
    sagemaker_session=session,
    role=sm_role,
)

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json",
    )
)

register_args = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge", "ml.m5.large"],
    transform_instances=["ml.m5.xlarge", "ml.m5.large"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status_param,
    model_metrics=model_metrics,
)

step_register = ModelStep(
    name=f"{pipeline_name}-register",
    step_args=register_args
)

#### Fail step
Add a Pipelines [FailStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.fail_step.FailStep) to stop the pipeline execution if the model performance metric doesn't meet the specified threshold. 

In [ ]:
# step_fail = FailStep(......................

In [ ]:
step_fail = FailStep(
    name=f"{pipeline_name}-fail",
    error_message=Join(on=" ", values=["Execution failed due to AUC Score >", test_score_threshold_param]),
)

#### Condition step
The condition step checks the model performance score and conditionally creates a model and registers it in the model registry, or stops and fails the pipeline execution.

In [ ]:
# cond_lte = ConditionGreaterThan(...........

In [ ]:
cond_lte = ConditionGreaterThan(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="classification_metrics.auc_score.value",
    ),
    right=test_score_threshold_param,
)

step_cond = ConditionStep(
    name=f"{pipeline_name}-check-test-score",
    conditions=[cond_lte],
    if_steps=[step_register],
    else_steps=[step_fail],
)

#### Construct the pipeline 


In [ ]:
# pipeline = Pipeline(
#     name=pipeline_name,....................

In [ ]:
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        process_instance_type_param,
        train_instance_type_param,
        train_instance_count_param,
        model_approval_status_param,
        test_score_threshold_param,
        input_s3_url_param,
    ],
    steps=[step_process, step_train, step_eval, step_cond],
    sagemaker_session=session,
)

In [ ]:
# Create a new or update existing Pipeline
pipeline.upsert(role_arn=sm_role.expr())

In [ ]:
pipeline_definition = json.loads(pipeline.describe()['PipelineDefinition'])
pipeline_definition

## Execute the pipeline
The following code starts an execution of the pipeline with the specified parameters.

In [ ]:
execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType=process_instance_type,
        TrainingInstanceType=train_instance_type,
        TrainingInstanceCount=train_instance_count,
        ModelApprovalStatus="PendingManualApproval",
        TestScoreThreshold=0.75,
        InputDataUrl=input_s3_url
    )
)

In [ ]:
# Un-comment this call if you want the notebook to wait until the pipeline's execution finished
execution.wait()
execution.list_steps()